In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
%matplotlib inline
from src.metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key' : 'user_id',
                     'product_id' : 'item_id'},
            inplace=True)
test_size_week = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_week]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_week]
data_train

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2488946,98,41944918665,684,945779,2,2.00,421,0.00,1705,98,0.0,0.0
2488947,98,41944918665,684,993617,2,2.00,421,0.00,1705,98,0.0,0.0
2488948,98,41944918665,684,1128647,2,2.00,421,0.00,1705,98,0.0,0.0
2488949,98,41944918665,684,9526886,2,0.60,421,0.00,1705,98,0.0,0.0


In [3]:
item_features = pd.read_csv('data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id' : 'item_id'}, inplace=True)

item_features

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,


In [4]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']
result

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."
2,5,"[913077, 1118028, 1386668]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108..."
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10..."
...,...,...
1986,2494,"[826265, 828393, 834303, 835236, 839147, 84036..."
1987,2497,"[906777, 996833, 1032967, 15830980, 15928068, ..."
1988,2498,"[1014185, 13417441, 993638, 906335, 991948, 10..."
1989,2499,"[834117, 844441, 999971, 15924955, 18148096, 8..."


In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity' : 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold' , ascending=False).head(5000).item_id.tolist()

In [7]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train,
                                 index='user_id', columns='item_id',
                                 values='quantity',
                                 aggfunc='count',
                                 fill_value = 0
                                 )

user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix)
user_item_matrix

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                      model.recommend(userid=userid_to_id[user],
                                     user_items=sparse_user_item, # на вход user-item matrix
                                     N=N,
                                     filter_already_liked_items=False,
                                     filter_items=None,
                                     recalculate_user=True)]
    
    return res

In [10]:
def scoring(result, model):
    result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
    return result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

In [11]:
%%time

model = AlternatingLeastSquares(factors=100,
                               regularization=0.001,
                               iterations=15,
                               calculate_training_loss=True,
                               num_threads=4)

model.fit(csr_matrix(user_item_matrix).T, # на вход user_item_matrix
         show_progress=True)

recs = model.recommend(userid=userid_to_id[2], # userid - id от 0 до N
                      user_items=csr_matrix(user_item_matrix).tocsr(),  # на вход user-item matrix
                      N=5, #кол-во рекомендаций
                      filter_already_liked_items=False, 
                      filter_items=None, 
                      recalculate_user=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 24.1 s, sys: 37.7 s, total: 1min 1s
Wall time: 13 s


In [28]:
%%time
scoring(result, model)

CPU times: user 38.9 s, sys: 1min 26s, total: 2min 5s
Wall time: 17.5 s


0.1495730788548468

In [13]:
no_popularity = item_features.groupby('department')['manufacturer'].unique().reset_index() 
no_popularity

,department,manufacturer
0,,[1]
1,AUTOMOTIVE,"[5474, 5]"
2,CHARITABLE CONT,"[2287, 2205]"
3,CHEF SHOPPE,"[3110, 3210, 2, 3116, 3172, 5565, 5111, 5423]"
4,CNTRL/STORE SUP,"[1871, 5879, 6011, 903]"
5,COSMETICS,"[373, 69, 958, 1189, 5014, 5009, 5123, 5072, 4..."
6,COUP/STR & MFG,"[2745, 2, 2232, 151, 882, 233, 76, 1543, 2650,..."
7,DAIRY DELI,"[1563, 2303]"
8,DELI,"[69, 4085, 1653, 3574, 3758, 454, 1858, 3730, ..."
9,DELI/SNACK BAR,"[332, 94]"


In [25]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    data_train['price'] = data_train['sales_value'] / np.maximum(data_train['quantity'], 1)
    price = data_train.groupby('item_id')['price'].nunique().reset_index() 

    cheap_products = price[price['price'] < price['price'].quantile(0.20)].item_id.tolist()
    data = data[~data['item_id'].isin(cheap_products)]
    # Уберем слишком дорогие товарыs
    
    expensive_products = price[price['price'] > price['price'].quantile(0.98)].item_id.tolist()
    data = data_train[~data_train['item_id'].isin(expensive_products)]
    # ...
    return data

def postfilter_items(user_id, recommednations):
    pass

In [26]:
pre_filter = prefilter_items(data_test)
pre_filter = pre_filter.groupby('user_id')['item_id'].unique().reset_index()
pre_filter.columns = ['user_id', 'actual']

<ipython-input-25-8c3034e31f99>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['price'] = data_train['sales_value'] / np.maximum(data_train['quantity'], 1)


In [29]:
%%time
scoring(pre_filter, model)

CPU times: user 43.3 s, sys: 1min 39s, total: 2min 22s
Wall time: 19.4 s


0.5419043792687828